# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('output_data/cities.csv')
cities_df.head()

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,albany,42.60,-73.97,84.99,54,60,5.61,US,1595809500
1,1,pevek,69.70,170.31,53.62,68,87,6.26,RU,1595809656
2,2,broome,42.25,-75.83,82.00,57,1,4.70,US,1595809769
3,3,dikson,73.51,80.55,37.94,90,90,14.97,RU,1595809770
4,4,zeya,53.75,127.27,73.40,70,85,4.32,RU,1595809771


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_df[["Lat", "Lon"]]

humidity = cities_df["Humidity"]

figure = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 250, point_radius = 4)
figure.add_layer(heatmap)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_cities_df = cities_df.loc[(cities_df["Max Temp"] <80) & (cities_df["Max Temp"]  >70) 
                                   & (cities_df["Windspeed"] < 10) & (cities_df["Cloudiness"] == 0)]
narrowed_cities_df

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
8,8,herat,34.50,62.00,78.80,27,0,9.17,AF,1595809777
20,20,portoferraio,42.81,10.31,73.99,75,0,5.32,IT,1595809795
27,27,agadir,30.42,-9.60,71.60,83,0,3.40,MA,1595809700
39,39,sari,36.56,53.06,77.00,88,0,4.70,IR,1595809819
60,60,chitral,35.85,71.79,75.00,42,0,1.81,PK,1595809849
68,68,lesozavodsk,45.48,133.42,76.41,71,0,2.48,RU,1595809860
99,99,alta floresta,-9.88,-56.09,74.46,44,0,4.18,BR,1595809899
148,148,ariquemes,-9.91,-63.04,75.52,46,0,1.83,BR,1595809961
218,218,ahmadnagar,19.08,74.73,73.15,87,0,8.52,IN,1595810047
219,219,behshahr,36.69,53.55,77.00,88,0,4.70,IR,1595810048


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrowed_cities_df[["City", "Country", "Lat", "Lon"]]
hotel_df["Hotel Name"] = ""
hotel_df.head()

/opt/anaconda3/envs/NewPythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Lat,Lon,Hotel Name
8,herat,AF,34.50,62.00,
20,portoferraio,IT,42.81,10.31,
27,agadir,MA,30.42,-9.60,
39,sari,IR,36.56,53.06,
60,chitral,PK,35.85,71.79,


In [13]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
# https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters

# https://maps.googleapis.com/maps/api/place/nearbysearch/json?key=[YOURAPIKEY]&location=-33.8670522,151.1957362&radius=5000&type=restaurant
    
counter = 0
hotel_result = []

# parameters{
#     "radius":5000
#     "types": "lodging"
#     "key": g_key
# }

for index, row in hotel_df.iterrows():
    lat= row["Lat"]
    lng= row["Lon"]
    
#     parameters["location"]= (f"{lat},{lng}")

    query_url = (f"{url}key={g_key}&location={lat},{lng}&radius=5000&type=lodging")
    
    try:
        time.sleep(1)
        response = requests.get(query_url).json
        
#         hotel_result.append(response)
        
#         hotel_df["Hotel Name"] = hotel_result
        
        counter += 1         
                 
                 
    except KeyError:
        print(f"No hotels found")
    
    if counter == 3 :
                 
                
        break
                 
print(response)

<bound method Response.json of <Response [200]>>


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
